<a href="https://colab.research.google.com/github/chesnokkk/Potholes_object_detection/blob/main/Tf_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/Colab_Notebooks/TF_object_detection

/content/drive/MyDrive/Colab_Notebooks/TF_object_detection


## Load model and test it

In [ ]:
!git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [ ]:
import os
os.environ['PYTHONPATH'] += ":/content/drive/MyDrive/Colab_Notebooks/TF_object_detection/models"

import sys
sys.path.append("/content/drive/MyDrive/Colab_Notebooks/TF_object_detection/models")

In [ ]:
!apt-get install protobuf-compiler python-lxml python-pil 
!pip install Cython pandas tf-slim lvis
!pip install tensorflow-object-detection-api

In [ ]:
%cd models/research

/content/drive/MyDrive/Colab_Notebooks/TF_object_detection/models/research


In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
pip install tensorflow-object-detection-api

In [ ]:
import os
os.environ['PYTHONPATH'] = '/content/drive/MyDrive/Colab_Notebooks/TF_object_detection/models/research:/content/drive/MyDrive/Colab_Notebooks/TF_object_detection/models/models/research/slim:' + os.environ['PYTHONPATH']

In [ ]:
!pip install tensorflow_io

     |████████████████████████████████| 23.1 MB 1.3 MB/s 


In [ ]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 8.5 MB/s 


In [ ]:
# Test if the Object Dectection API is working correctly
!python object_detection/builders/model_builder_tf2_test.py 

2022-01-14 18:12:59.789153: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0114 18:13:00.209709 139626765748096 model_builder.py:1100] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.71s
I0114 18:13:00.498717 139626765748096 test_util.py:2309] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.71s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.89s
I0114 18:13:01.387632 139626765748096 test_util.py:2309] time(__main__.ModelBuilderTF2Test.test_cr

In [ ]:
import tensorflow as tf

print(tf.__version__)



2.7.0


## Downloading dataset

In [ ]:
%%bash
cd /content/drive/MyDrive/Colab_Notebooks/TF_object_detection
kaggle datasets download -d yamqwe/potholes-object-detection

100%|##########| 334M/334M [00:03<00:00, 118MB/s]


In [ ]:
%cd /content/drive/MyDrive/Colab_Notebooks/TF_object_detection

/content/drive/MyDrive/Colab_Notebooks/TF_object_detection


In [ ]:
import os
IMAGE_PATH = '/content/drive/MyDrive/Colab_Notebooks/TF_object_detection/potholes-object-detection'

if not os.path.exists(IMAGE_PATH):
    !mkdir -p {IMAGE_PATH}
    !mv potholes-object-detection.zip {IMAGE_PATH}
    %cd {IMAGE_PATH}
    !unzip potholes-object-detection.zip

In [ ]:
ls

models/  potholes-object-detection.zip  Tf_object_detection.ipynb


In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

### Create label map

In [ ]:
LABEL_MAP_PATH = '/content/drive/MyDrive/Colab_Notebooks/TF_object_detection/potholes-object-detection/annotations/label_map.pbtxt'

In [ ]:
labels = [{'name':'pothole', 'id':1}]

with open(LABEL_MAP_PATH, 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

### Move annotations in image folder and train/test split

In [ ]:
#Info https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html
%%bash

cd /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/potholes-object-detection/annotations
mv * /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/potholes-object-detection/images
cd /content/drive/MyDrive/Colab_Notebooks/TF_object_detection

mkdir scripts
cd scripts
wget https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/d0e545609c5f7f49f39abc7b6a38cec3/partition_dataset.py
python partition_dataset.py -x -i /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/potholes-object-detection/images -r 0.1

### Create TF_records

In [ ]:
%%bash

cd /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/scripts
wget https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py

--2022-01-14 10:33:41--  https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py
Resolving tensorflow-object-detection-api-tutorial.readthedocs.io (tensorflow-object-detection-api-tutorial.readthedocs.io)... 104.17.33.82, 104.17.32.82, 2606:4700::6811:2152, ...
Connecting to tensorflow-object-detection-api-tutorial.readthedocs.io (tensorflow-object-detection-api-tutorial.readthedocs.io)|104.17.33.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6410 (6.3K) [text/x-python]
Saving to: ‘generate_tfrecord.py.1’

     0K ......                                                100% 3.80M=0.002s

2022-01-14 10:33:41 (3.80 MB/s) - ‘generate_tfrecord.py.1’ saved [6410/6410]



In [ ]:
%cd /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/scripts

/content/drive/MyDrive/Colab_Notebooks/TF_object_detection/scripts


In [ ]:
!python generate_tfrecord.py -x /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/potholes-object-detection/images/train -l /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/potholes-object-detection/annotations/label_map.pbtxt -o /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/potholes-object-detection/images/train.record
!python generate_tfrecord.py -x /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/potholes-object-detection/images/test -l /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/potholes-object-detection/annotations/label_map.pbtxt -o /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/potholes-object-detection/images/test.record

Successfully created the TFRecord file: /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/potholes-object-detection/images/train.record
Successfully created the TFRecord file: /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/potholes-object-detection/images/test.record


## Download model




In [ ]:
%cd /content/drive/MyDrive/Colab_Notebooks/TF_object_detection
if not os.path.exists('downloading_models'):
    !mkdir downloading_models
%cd downloading_models

/content/drive/MyDrive/Colab_Notebooks/TF_object_detection
/content/drive/My Drive/Colab_Notebooks/TF_object_detection/downloading_models


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz
!tar -xvzf centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz

centernet_mobilenetv2_fpn_od/
centernet_mobilenetv2_fpn_od/model.tflite
centernet_mobilenetv2_fpn_od/saved_model/
centernet_mobilenetv2_fpn_od/saved_model/saved_model.pb
centernet_mobilenetv2_fpn_od/saved_model/variables/
centernet_mobilenetv2_fpn_od/saved_model/variables/variables.index
centernet_mobilenetv2_fpn_od/saved_model/variables/variables.data-00000-of-00001
centernet_mobilenetv2_fpn_od/saved_model/assets/
centernet_mobilenetv2_fpn_od/label_map.txt
centernet_mobilenetv2_fpn_od/pipeline.config
centernet_mobilenetv2_fpn_od/checkpoint/
centernet_mobilenetv2_fpn_od/checkpoint/ckpt-301.index
centernet_mobilenetv2_fpn_od/checkpoint/checkpoint
centernet_mobilenetv2_fpn_od/checkpoint/ckpt-301.data-00000-of-00001


In [ ]:
#manualy change pipline.config

In [ ]:
#copy training script into script folder
!cp /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/models/research/object_detection/model_main_tf2.py /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/scripts

In [ ]:
%%bash

cd /content/drive/MyDrive/Colab_Notebooks/TF_object_detection/scripts
python model_main_tf2.py \
    --model_dir=/content/drive/MyDrive/Colab_Notebooks/TF_object_detection/downloading_models/centernet_mobilenetv2_fpn_od \
    --pipeline_config_path=/content/drive/MyDrive/Colab_Notebooks/TF_object_detection/downloading_models/centernet_mobilenetv2_fpn_od/pipeline.config \
    --checkpoint_dir=/content/drive/MyDrive/Colab_Notebooks/TF_object_detection/downloading_models/centernet_mobilenetv2_fpn_od/checkpoint/checkpoint \
    --num_train_steps=10 \
    --alsologtostderr

9420800/9406464 [==============================] - 0s 0us/step


2022-01-14 18:13:56.389873: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
W0114 18:13:57.125869 140343179253632 model_lib_v2.py:1090] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0114 18:13:57.126230 140343179253632 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0114 18:13:57.126355 140343179253632 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting train_steps: 10
I0114 18:13:57.126456 140343179253632 config_util.py:552] Maybe overwriting train_steps: 10
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0114 18:13:57.126565 140343179253632 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0114 18:13:57.126689 140343179253632 model_lib_v2.py:1111] Expected number of evaluation epochs is 1,

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
